In [20]:
from bs4 import BeautifulSoup # magical tool for parsing html data
import requests
import pandas as pd
from pandas import DataFrame as df

In [21]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

In [22]:
list_url = ['https://cordis.europa.eu/project/id/BRRT975001', 
            'https://cordis.europa.eu/project/id/G1RT-CT-2002-05078', 
            'https://cordis.europa.eu/project/id/291870', 
            'https://cordis.europa.eu/project/id/730238',
            'https://cordis.europa.eu/project/id/ICA1-CT-2002-60042',
            'https://cordis.europa.eu/project/id/MAS3980182', 
            'https://cordis.europa.eu/project/id/IST-1999-10337', 
            'https://cordis.europa.eu/project/id/ENK6-CT-1999-00010', 
            'https://cordis.europa.eu/project/id/FP4_23802', 
            'https://cordis.europa.eu/project/id/11745', 
            'https://cordis.europa.eu/project/id/ENK6-CT-2002-20651', 
            'https://cordis.europa.eu/project/id/262371',
            'https://cordis.europa.eu/project/id/280855',
            'https://cordis.europa.eu/project/id/608921',
            'https://cordis.europa.eu/project/id/312845',
            'https://cordis.europa.eu/project/id/641999',
            'https://cordis.europa.eu/project/id/689648',
            'https://cordis.europa.eu/project/id/690133',
            'https://cordis.europa.eu/project/id/730127',
            'https://cordis.europa.eu/project/rcn/216083/en',
            'https://cordis.europa.eu/project/id/776642',
            'https://cordis.europa.eu/project/id/820852',
            'https://cordis.europa.eu/project/id/776811',
            'https://cordis.europa.eu/project/id/731166']

In [23]:
sites = []
for url in list_url :
    print(url)
    site = requests.get(url)
    sites.append(site)

https://cordis.europa.eu/project/id/BRRT975001
https://cordis.europa.eu/project/id/G1RT-CT-2002-05078
https://cordis.europa.eu/project/id/291870
https://cordis.europa.eu/project/id/730238
https://cordis.europa.eu/project/id/ICA1-CT-2002-60042
https://cordis.europa.eu/project/id/MAS3980182
https://cordis.europa.eu/project/id/IST-1999-10337
https://cordis.europa.eu/project/id/ENK6-CT-1999-00010
https://cordis.europa.eu/project/id/FP4_23802
https://cordis.europa.eu/project/id/11745
https://cordis.europa.eu/project/id/ENK6-CT-2002-20651
https://cordis.europa.eu/project/id/262371
https://cordis.europa.eu/project/id/280855
https://cordis.europa.eu/project/id/608921
https://cordis.europa.eu/project/id/312845
https://cordis.europa.eu/project/id/641999
https://cordis.europa.eu/project/id/689648
https://cordis.europa.eu/project/id/690133
https://cordis.europa.eu/project/id/730127
https://cordis.europa.eu/project/rcn/216083/en
https://cordis.europa.eu/project/id/776642
https://cordis.europa.eu/pr

In [ ]:
len(sites)

In [ ]:
title = []
acronyme = []
#contenu = []
#funded = []
ID = []
start = []
end = []
overall = []
EU = []
coord = []
coordcountry = []
names = []
countries = []

for s in sites : 
    #Create a soup object/ CORDIS
    
    if s.ok:
        soup = BeautifulSoup(s.text, 'html.parser') #parser


        #contenu .append soup.find("p",{'class':'c-article__text'}).contents
        #comment coder quand par exmple là - le "contenu est vide" ? comment faire afficher none/ nul ? sans le rapport d'erreur

        title.append(soup.find("h1",{'class': 'c-header-project__title ng-star-inserted'}).contents[0]) #titre
        acronyme.append(soup.find('div', {'class':'c-project-info__acronym'}).contents[0]) #Project acronyme
        #contenu.append(soup.find("p",{'class':'c-article__text'}).contents[1]) #contenu de la page
        #funded.append(soup.find('li', {'class':'ng-star-inserted'}).contents[0]) #funded under
        ID.append(soup.find('div', {'class':'c-project-info__id'}).contents[0]) #Grant agreement ID
        start.append(soup.find('div',{'class':'col-6'}).contents[1]) #Start date
        end.append(soup.find('div',{'class':'col-6 t-text-align-right'}).contents[1]) #End date
        overall.append(soup.find('div',{'class':'c-project-info__overall'}).contents[1] )#Overall budget
        EU.append(soup.find('div',{'class':'c-project-info__eu ng-star-inserted'}).contents[1]) #EU contribution
        coord.append(soup.find('p',{'class':'coordinated coordinated-name ng-star-inserted'}).contents[0]) #Coordinated by
        coordcountry.append(soup.find('p',{'class':'coordinated coordinated-name ng-star-inserted'}).contents[5]) #Coordinator country
        names.append([n.contents[0] for n in soup.find_all('div',{'class':'c-part-info__title'})])
        countries.append([n.contents[1].replace('','') for n in soup.find_all('div',{'class':'c-part-info__country c-part-info__country-alt ng-star-inserted'})])


In [ ]:
for i in range(0,len(names)):
    print(len(names[i]),len(countries[i]))

In [ ]:
df = pd.DataFrame([],columns=['titre','acronyme','ID','début','fin','budget total', 'participation EU', 'coordonateur', 'pays du coord', 'participants', 'pays des participants'])
projet = pd.DataFrame([title[0],acronyme[0],ID[0],start[0],end[0],overall[0],EU[0],coord[0],coordcountry[0]])   
projet = projet.T
projet = pd.concat([projet]*len(names[0]))
df = pd.DataFrame()
for i in range(0,len(title)):
    projet = pd.DataFrame([title[i],acronyme[i],ID[i],start[i],end[i],overall[i],EU[i],coord[i],coordcountry[i]])
    projet = projet.T
    projet = pd.concat([projet]*len(names[i]),axis = 0) # repete la ligne du projet len(names[i]) fois
    projet['participants'] = names[i]
    projet['pays des participants'] = [''] + countries[i]
    
    df = pd.concat([df,projet],axis = 0) # concatene les datframes dans le sens des lignes (axis = 0 et pas 1)
df.columns =['titre','acronyme','ID','début','fin','budget total', 'participation EU', 'coordonateur', 'pays du coord', 'participants', 'pays des participants']        
df.reset_index(inplace = True,drop=True) # reindice le dataframe


In [ ]:
# conversion des date en string en objet date (pratique pour excel) / passage ne fonctionne pas
df['début'] = df['début'].apply(lambda x : pd.to_datetime(x).date())
df['fin'] = df['fin'].apply(lambda x : pd.to_datetime(x).date())

In [ ]:
df.to_excel('scrapping_cordis_test.xls')